Data originally from this source: https://www.kaggle.com/datasets/ealtman2019/credit-card-transactions

In [1]:
import pandas as pd
import numpy as np
import datetime
import mysql.connector

from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [2]:
# Create connection to mysql database
credit_card_transactions_db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password=""
)

print(credit_card_transactions_db) 

Let's begin by reading in the contents of both tables to pandas dataframes, and then using Python to explore and clean the data. We will only consider non fraudulent transactions no older than Jan 1, 2012.

In [3]:
query1 = """
SELECT 
  User,
  Amount,
  `Use Chip`,
  `Merchant Name`,
  `Merchant State`,
  MCC,
  Year,
  DATE_ADD(DATE_ADD(MAKEDATE(Year, 1), INTERVAL (Month)-1 MONTH), INTERVAL (Day)-1 DAY) AS Date,
  TIMESTAMP(DATE_ADD(DATE_ADD(MAKEDATE(Year, 1), INTERVAL (Month)-1 MONTH), INTERVAL (Day)-1 DAY),CONVERT(TIME, Time)) AS Datetime
FROM CreditCardTransactions.transactions
WHERE 
  `Is Fraud?` = "No"
  AND Year >= 2012;
"""
query2 = """
SELECT
  id AS User,
  Gender,
  `Current Age`,
  `Retirement Age`,
  `Yearly Income - Person`,
  `Total Debt`,
  `FICO Score`,
  `Num Credit Cards`
FROM CreditCardTransactions.users;
"""
transactions_df = pd.read_sql(query1, credit_card_transactions_db)
users_df = pd.read_sql(query2, credit_card_transactions_db)
credit_card_transactions_db.close()

In [4]:
transactions_df.head(3)

,User,Amount,Use Chip,Merchant Name,Merchant State,MCC,Year,Date,Datetime
0,1,$7.93,Swipe Transaction,4060646732831064559,CA,5411,2012,2012-01-01,2012-01-01 14:15:00
1,1,$3.46,Swipe Transaction,-34551508091458520,CA,5912,2012,2012-01-02,2012-01-02 12:57:00
2,1,$68.79,Swipe Transaction,4060646732831064559,CA,5411,2012,2012-01-02,2012-01-02 13:31:00


From a quick glance at a sample of the contents of the data set, we can see that *transactions_df* appears to contain a list of card-based transactions with commonly associated details such as the date and time at which the transaction occurred, amount of the transaction, etc., but also information on the Merchant, including Merchant MCC which corresponds with the broad category of service or product offered by that particular business. We will explore the classification of the  shortly.

In [5]:
users_df.head(3)

,User,Gender,Current Age,Retirement Age,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
0,1,Female,53,66,$59696,$127613,787,5
1,2,Female,53,68,$77254,$191349,701,5
2,3,Female,81,67,$33483,$196,698,5


This data set contains information on the customer, including income and credit score.

Let's begin working on the *transactions* dataset.

In [6]:
transactions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1227570 entries, 0 to 1227569
Data columns (total 9 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   User            1227570 non-null  int64         
 1   Amount          1227570 non-null  object        
 2   Use Chip        1227570 non-null  object        
 3   Merchant Name   1227570 non-null  int64         
 4   Merchant State  1227570 non-null  object        
 5   MCC             1227570 non-null  int64         
 6   Year            1227570 non-null  int64         
 7   Date            1227570 non-null  object        
 8   Datetime        1227570 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 84.3+ MB


First, we'll convert the Amount column to a numeric type, then we'll replace the values in values in the MCC column with their corresponding business classification.

In [7]:
transactions_df['Amount'] = transactions_df['Amount'].str.replace(
    '$',
    ''
).astype(float)

transactions_df['User'] = transactions_df['User'].astype(object)
transactions_df['Merchant Name'] = transactions_df['Merchant Name'].astype(object)
transactions_df['Merchant State'] = pd.Categorical(transactions_df['Merchant State'])
transactions_df['Year'] = pd.Categorical(transactions_df['Year'])

<ipython-input-7-2c7c96baae7a>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  transactions_df['Amount'] = transactions_df['Amount'].str.replace(


In [8]:
# For reference
mcc_dict = {
    '0001-1499': 'Agricultural Services',
    '1500-2999': 'Contracted Services',
    '3000-3499': 'Travel Services',
    '4000-4799': 'Transportation Services',
    '4800-4999': 'Utility Services',
    '5000-5599': 'Retail Outlet Services',
    '5600-5699': 'Clothing Stores',
    '5700-7299': 'Miscellaneous Stores',
    '7300-7999': 'Business Services',
    '8000-8999': 'Professional Services and Membership Organizations',
    '9000-9999': 'Government Services'
    }


In [9]:
bins = [1, 1500, 3000, 4000, 4800, 5000, 5600, 5700, 7300, 8000, 9000, 9999]
labels = [
    'Agricultural Services',
    'Contracted Services',
    'Travel Services',
    'Transportation Services',
    'Utility Services',
    'Retail Outlet Services',
    'Clothing Stores',
    'Miscelleanous Stores',
    'Business Services',
    'Professional Services and Membership Organizations',
    'Government Services'
]

transactions_df['MCC Category'] = pd.cut(
    transactions_df.MCC, 
    bins=bins, 
    labels=labels,  
    right=False, 
    include_lowest=True
    )

In [10]:
transactions_df.isna().sum()

User              0
Amount            0
Use Chip          0
Merchant Name     0
Merchant State    0
MCC               0
Year              0
Date              0
Datetime          0
MCC Category      0
dtype: int64

In [11]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   User                    2000 non-null   int64 
 1   Gender                  2000 non-null   object
 2   Current Age             2000 non-null   int64 
 3   Retirement Age          2000 non-null   int64 
 4   Yearly Income - Person  2000 non-null   object
 5   Total Debt              2000 non-null   object
 6   FICO Score              2000 non-null   int64 
 7   Num Credit Cards        2000 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 125.1+ KB


In [12]:
users_df['User'] = users_df['User'].astype(object)

lst = ['Yearly Income - Person', 'Total Debt']
for i in lst:
    users_df[i] = users_df[i].str.replace(
        '$',
        ''
    ).astype(int)

users_df['Debt/Income'] = users_df['Total Debt']/users_df['Yearly Income - Person']

users_df['Gender'] = np.where(
    users_df['Gender']=='Male',
    0,
    1
)

<ipython-input-12-55451ff73c7d>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  users_df[i] = users_df[i].str.replace(


In [13]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User                    2000 non-null   object 
 1   Gender                  2000 non-null   int64  
 2   Current Age             2000 non-null   int64  
 3   Retirement Age          2000 non-null   int64  
 4   Yearly Income - Person  2000 non-null   int64  
 5   Total Debt              2000 non-null   int64  
 6   FICO Score              2000 non-null   int64  
 7   Num Credit Cards        2000 non-null   int64  
 8   Debt/Income             2000 non-null   float64
dtypes: float64(1), int64(7), object(1)
memory usage: 140.8+ KB


In [14]:
transaction_totals = transactions_df.groupby(['User', 'MCC Category']).sum()['Amount'].reset_index()

transaction_volumes = transactions_df.groupby(['User', 'MCC Category']).count()['Amount'].reset_index()

In [15]:
transaction_agg_by_user = pd.merge(
    transaction_totals,
    transaction_volumes,
    how='left',
    on=['User', 'MCC Category']
)

In [16]:
transaction_agg_by_user.rename(
    columns={
        'Amount_x':'TotalSum',
        'Amount_y':'TotalVol'
    },
    inplace=True
)

In [17]:
# Create new dataframe from amount spent by user per MCC
main_df = pd.merge(
    transaction_agg_by_user,
    users_df,
    on='User',
    how='right'
)

In [18]:
# Drop customers with no conducted transactions
main_df = main_df[~main_df['TotalVol'].isnull()]

In [19]:
# Define K-means model
kmeans_model = KMeans(init='k-means++',  max_iter=400, random_state=42)

In [20]:
main_df.head()

,User,MCC Category,TotalSum,TotalVol,Gender,Current Age,Retirement Age,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Debt/Income
0,1.0,Agricultural Services,0.00,0.0,1,53,66,59696,127613,787,5,2.137714
1,1.0,Contracted Services,163.06,1.0,1,53,66,59696,127613,787,5,2.137714
2,1.0,Travel Services,40197.70,243.0,1,53,66,59696,127613,787,5,2.137714
3,1.0,Transportation Services,695.83,17.0,1,53,66,59696,127613,787,5,2.137714
4,1.0,Utility Services,46069.11,358.0,1,53,66,59696,127613,787,5,2.137714


In [21]:
# Train the model
kmeans_model.fit(
    main_df[['TotalSum', 'TotalVol', 'Gender', 'Current Age', 'Yearly Income - Person', 'Total Debt']]
)

KMeans(max_iter=400, random_state=42)

In [22]:
# Create the K means model for different values of K
def try_different_clusters(K, data):
       
    cluster_values = list(range(1, K+1))
    inertias=[]
    
    for c in cluster_values:
        model = KMeans(
            n_clusters=c,
            init='k-means++',
            max_iter=400,
            random_state=60)
        model.fit(data)
        inertias.append(model.inertia_)
    
    return inertias

In [23]:
outputs = try_different_clusters(
    12, 
    main_df[['TotalSum', 'TotalVol', 'Gender', 'Current Age', 'Yearly Income - Person', 'Total Debt']]
    )
distances = pd.DataFrame({"clusters": list(range(1, 13)),"sum of squared distances": outputs})

In [24]:
# Finding optimal number of clusters k
figure = go.Figure()
figure.add_trace(go.Scatter(x=distances["clusters"], y=distances["sum of squared distances"]))

figure.update_layout(xaxis = dict(tick0 = 1,dtick = 1,tickmode = 'linear'),                  
                  xaxis_title="Number of clusters",
                  yaxis_title="Sum of squared distances",
                  title_text="Finding optimal number of clusters using elbow method")
figure.show()

In [25]:
kmeans_model_new = KMeans(
    n_clusters=4,
    init='k-means++',
    max_iter=400,
    random_state=60
    )

kmeans_model_new.fit_predict(
    main_df[['TotalSum', 'TotalVol', 'Gender', 'Current Age', 'Yearly Income - Person', 'Total Debt']]
)

array([1, 1, 1, ..., 2, 2, 2], dtype=int32)

In [26]:
# Create data arrays
cluster_centers = kmeans_model_new.cluster_centers_
data = np.expm1(cluster_centers)
points = np.append(data, cluster_centers, axis=1)

<ipython-input-26-e030eff150c2>:3: RuntimeWarning:

overflow encountered in expm1



In [27]:
# Add clusters column to main df
points = np.append(points, [[0], [1], [2], [3]], axis=1)
main_df["clusters"] = kmeans_model_new.labels_

In [28]:
main_df.groupby('clusters').mean()[['TotalSum', 'TotalVol', 'Current Age', 'Yearly Income - Person', 'Total Debt']]

,TotalSum,TotalVol,Current Age,Yearly Income - Person,Total Debt
clusters,,,,,
0,14866.842734,414.081616,47.871632,39858.301902,30462.290016
1,11957.497081,298.271380,36.840365,52286.737742,96792.403649
2,29124.598629,358.653226,42.612903,124988.233871,233333.088710
3,234896.819000,4986.775000,54.350000,60155.212500,70448.387500


In [29]:
main_df

,User,MCC Category,TotalSum,TotalVol,Gender,Current Age,Retirement Age,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Debt/Income,clusters
0,1.0,Agricultural Services,0.00,0.0,1,53,66,59696,127613,787,5,2.137714,1
1,1.0,Contracted Services,163.06,1.0,1,53,66,59696,127613,787,5,2.137714,1
2,1.0,Travel Services,40197.70,243.0,1,53,66,59696,127613,787,5,2.137714,1
3,1.0,Transportation Services,695.83,17.0,1,53,66,59696,127613,787,5,2.137714,1
4,1.0,Utility Services,46069.11,358.0,1,53,66,59696,127613,787,5,2.137714,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,217.0,Clothing Stores,0.00,0.0,1,24,66,101191,290730,659,1,2.873082,2
2343,217.0,Miscelleanous Stores,812.51,23.0,1,24,66,101191,290730,659,1,2.873082,2
2344,217.0,Business Services,450.25,8.0,1,24,66,101191,290730,659,1,2.873082,2
2345,217.0,Professional Services and Membership Organizat...,0.00,0.0,1,24,66,101191,290730,659,1,2.873082,2


In [30]:
transactions_df.to_csv('transactions-cleaned-data.csv')
users_df.to_csv('users-cleaned-data.csv')
main_df.to_csv('customer-spending.csv')